# POS tagger model

## Load data from JSON

In [4]:
import numpy as np
import json
import os
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
import pickle
import pandas as pd

In [ ]:
files = [os.path.join('json/gc/webtext/json',x) for x in os.listdir('json/gc/webtext/json')]
corpus = []

for x in files:
    with open(x,'r',encoding='utf-16') as f:
#         print('opening {0}'.format(x))
        doc = json.load(f)
        for i in doc.keys():
#             print("key {0} of document {1}".format(i,x))
            try:
#                 print(doc[i][0][0])
                if (len(doc[i][0][0])<200  and len(i) >0): #150 190 gave same max len
                       corpus.append(doc[i][0][0])  #append each sentence to corpus
            except:
# #                 print(doc[i])
                continue

In [ ]:


files = [os.path.join('json/gc/books',x) for x in os.listdir('json/gc/books')]
for x in files:
    with open(x,'r',) as f:
#         print('opening {0}'.format(x))
        doc = json.load(f)
#         for i in doc.keys():
        doc = doc ['ps']
        for i in doc:
            if len(i) <200 and len(i) >0:
                corpus.append(i)
#                 print(i)
            
%reset_selective -f doc
#     break

In [2]:
file_url = '../major8sem/Dataset/dataset_cs.csv'

In [6]:
df = pd.read_csv(file_url)
df = df[["tags","words"]]
corpus = []
sentence = []
for tag,word in zip(df["tags"],df["words"]):
    if(tag!='.'):
        sentence.append((tag,word))
    else:
        corpus.append(sentence)
        sentence = []
        

In [7]:
print(len(corpus))

59668


In [8]:
sentences = []
sentence_tags = []

for sentence in corpus:
    x=[]
    y=[]
    for word in sentence:
        x.append(word[1])
        y.append(word[0])
    if len(x) > 0:
        sentences.append(x)
        sentence_tags.append(y)
# sentences = sentences[:8000]
# sentence_tags = sentence_tags[:8000]
print("Sample sentence: ",sentences[10])
print("Sample sentence tags: ",sentence_tags[10])

Sample sentence:  ['उत्पादन', ',', 'वित्तीय', 'स्थिति', ',', 'वातावरण', 'को', 'कुरो', ',', 'बजार', 'उपलब्धता', 'आदि', 'ले', 'ऊनी', 'गलैंचा', 'को', 'निकासी', 'प्रभावित', 'पार्ने', 'बताउँदै', 'अध्यक्ष', 'श्री', 'श्रेष्ठ', 'ले', 'गलैंचा', 'धुवाई', 'मेशिन', 'हरू', 'ल्याएर', 'गलैंचा', 'हरू', 'को', 'धुवाई', 'नेपालमै', 'गरिना', 'ले', 'गलैंचा', 'व्यवसाय', 'सँग', 'सम्बन्धित', 'विदेशी', 'उद्यमी', 'हरू', 'लाई', 'प्रतिकूल', 'असर', 'पारे', 'को', 'ले', 'ती', 'उद्यमी', 'हरू', 'ले', 'नेपाल', 'मा', 'धुवाइ', 'गरिए', 'का', 'गलैंचा', 'हरू', 'प्रदूषित', 'हुने', 'हल्ला', 'गर्न', 'थाले', 'का', 'छन्', '।']
Sample sentence tags:  ['NN', 'YM', 'JX', 'NN', 'YM', 'NN', 'IKM', 'NN', 'YM', 'NN', 'NN', 'RR', 'IE', 'DDX', 'NN', 'IKM', 'NN', 'JX', 'VN', 'VDX', 'NN', 'NN', 'NP', 'IE', 'NN', 'NN', 'NN', 'IH', 'VQ', 'NN', 'IH', 'IKM', 'NN', 'NN', 'VI', 'IE', 'NN', 'NN', 'II', 'JX', 'JX', 'NN', 'IH', 'IA', 'JX', 'NN', 'VE', 'IKM', 'IE', 'DDX', 'NN', 'IH', 'IE', 'NP', 'II', 'NN', 'VE', 'IKO', 'NN', 'IH', 'JX', 'VN', 'NN', 

In [9]:
print("Tagged_sentences",len(sentence_tags))

Tagged_sentences 59375


In [10]:
print("Sample sentence: ",sentences[112])
print("Sample sentence tags: ",sentence_tags[112])

Sample sentence:  ['हाइटी', 'का', 'अपदस्थ', 'राष्ट्रपति', 'आरिष्टिड', 'सँग', 'अमेरिकी', 'राष्ट्रपति', 'बुश', 'को', 'यो', 'पहिलो', 'भेट', 'हो', '।']
Sample sentence tags:  ['NP', 'IKO', 'JX', 'NN', 'NP', 'II', 'JX', 'NN', 'NP', 'IKM', 'DDX', 'MOM', 'NN', 'VVYN1', 'YF']


## Convert Labels to numbers

In [11]:
labels = set()
for sentence in sentence_tags:
    for tag in sentence:
        labels.add(tag)
        
tag2index = {t: i + 1 for i, t in enumerate(list(labels))}
tag2index
print("Total number of tags: ",len(labels))

Total number of tags:  116


In [ ]:
# ##Load tag2index

# with open('withMaskingtag2index.pickle','rb') as f:
#     tag2index = pickle.load(f)

In [12]:
tag2index['-PAD-'] = 0


In [13]:
def tagsent2int(sent_tag):
    return [tag2index[tag] for tag in sent_tag]

sentence_tags = list(map(tagsent2int,sentence_tags))
print("Sample sentence: ",sentences[10])
print("Sample sentence tags: ",sentence_tags[10])

Sample sentence:  ['उत्पादन', ',', 'वित्तीय', 'स्थिति', ',', 'वातावरण', 'को', 'कुरो', ',', 'बजार', 'उपलब्धता', 'आदि', 'ले', 'ऊनी', 'गलैंचा', 'को', 'निकासी', 'प्रभावित', 'पार्ने', 'बताउँदै', 'अध्यक्ष', 'श्री', 'श्रेष्ठ', 'ले', 'गलैंचा', 'धुवाई', 'मेशिन', 'हरू', 'ल्याएर', 'गलैंचा', 'हरू', 'को', 'धुवाई', 'नेपालमै', 'गरिना', 'ले', 'गलैंचा', 'व्यवसाय', 'सँग', 'सम्बन्धित', 'विदेशी', 'उद्यमी', 'हरू', 'लाई', 'प्रतिकूल', 'असर', 'पारे', 'को', 'ले', 'ती', 'उद्यमी', 'हरू', 'ले', 'नेपाल', 'मा', 'धुवाइ', 'गरिए', 'का', 'गलैंचा', 'हरू', 'प्रदूषित', 'हुने', 'हल्ला', 'गर्न', 'थाले', 'का', 'छन्', '।']
Sample sentence tags:  [61, 56, 70, 61, 56, 61, 48, 61, 56, 61, 61, 82, 67, 39, 61, 48, 61, 70, 52, 86, 61, 61, 37, 67, 61, 61, 61, 81, 95, 61, 81, 48, 61, 61, 116, 67, 61, 61, 30, 70, 70, 61, 81, 114, 70, 61, 101, 48, 67, 39, 61, 81, 67, 37, 30, 61, 101, 10, 61, 81, 70, 52, 61, 116, 101, 10, 14, 4]


## Load word2vec model

In [17]:
import gensim.models.word2vec as w2v
nepw2v=w2v.Word2Vec.load('../major8sem/nepw2vmodel')

In [ ]:
#Load tokenizer

# with open('withMaskingTokenizer.pickle','rb') as f:
#     tokenizer = pickle.load(f)

In [19]:
tokenizer = Tokenizer(lower=False, oov_token='-OOV-')
tokenizer.fit_on_texts(sentences)
vocab_size = len(tokenizer.word_index) + 1
encoded_docs = tokenizer.texts_to_sequences(sentences)
# max_length = len(max(sentences,key=len))
max_length = 197
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
padded_tags = pad_sequences(sentence_tags,maxlen=max_length, padding='post')

In [20]:
tokenizer.word_index['-PAD-'] = 0

In [ ]:

# with open('withMaskingtag2index.pickle','wb') as f:
#     pickle.dump(tag2index,f)
# with open('nepw2v')

In [22]:
# create a weight matrix for words in training docs
embedding_matrix = np.zeros((vocab_size, nepw2v.wv.vector_size))
for word, i in tokenizer.word_index.items():
#     print(word)
    try:
        embedding_vector = nepw2v.wv.get_vector(word)
    except:
        embedding_vector = None
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [ ]:

# with open('nepw2vembeddingmatrix.pickle','wb') as f:
#     pickle.dump(embedding_matrix,f)
# with open('withMaskingTokenizer.pickle','wb') as f:
#     pickle.dump(tokenizer,f)

## Train Test split

In [23]:
from sklearn.model_selection import train_test_split
# skf = StratifiedKFold(n_splits=3,shuffle=True)
train_sentences,test_sentences,train_sentence_tags,test_sentence_tags = train_test_split(padded_docs,padded_tags,test_size=0.2)

## Train

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, InputLayer, Dropout, Bidirectional, TimeDistributed, Embedding, Activation,Masking
from keras.optimizers import Adam
 
model = Sequential()
model.add(InputLayer(input_shape=(max_length,)))
model.add(Embedding(vocab_size,300,weights=[embedding_matrix],input_length=max_length,trainable=False))
model.add(Masking())
model.add(Bidirectional(LSTM(256, return_sequences=True,drop)))
model.add(Dropout(0.2))
model.add(TimeDistributed(Dense(len(tag2index))))
model.add(Activation('softmax'))
 
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(0.001),
              metrics=['accuracy'])
 
model.summary()

In [ ]:
model.fit(train_sentences,to_categorical(train_sentence_tags),batch_size=256,epochs=10,validation_split=0.2)

## Save Model

In [ ]:
from keras.models import save_model
save_model(model,'withMasking.h5')

## Load Model

In [ ]:
from keras.models import load_model
model = load_model('withMasking.h5')

##  Test model

In [ ]:
test_sentence_tags.shape

In [ ]:
# model.evaluate(test_sentences,to_categorical(test_sentence_tags))
# model.evalutate(sentences,to_categorical(sentence_tags))
model.predict(np.asarray([padded_docs[0]]))

In [ ]:
def logits_to_sentence(sequences, index):
    token_sequences = []
    for categorical_sequence in sequences:
        token_sequence = []
        for categorical in categorical_sequence:
#             try:
            token_sequence.append(index[categorical])

        token_sequences.append(token_sequence)
    return token_sequences[0]

In [ ]:
# with open('withMaskingTokenizer.pickle','wb') as f:
#     pickle.dump(tokenizer,f)

In [ ]:
test = np.asarray([padded_docs[127]])
test_tag = np.asarray([padded_tags[127]])


In [ ]:
predicted =model.predict(test)

In [ ]:
# with open('withMaskingTokenizer.pickle','rb') as f:
#     tokenizer = pickle.load(f)
    

# with open('withMaskingtag2index.pickle','rb') as f:
#     tag2index = pickle.load(f)

In [ ]:
tokenizer.word_index['-00V-'] = 0
p_sent_tag = logits_to_tokens(predicted, {i: t for t, i in tag2index.items()})
a_sent_tag = logits_to_sentence(test_tag, {i: t for t, i in tag2index.items()})
p_sent = logits_to_sentence(test, {i: t for t, i in tokenizer.word_index.items()})

In [ ]:
for word,tag in zip(p_sent,a_sent_tag):
    if word!='-00V-' :
        print((word,tag))

In [ ]:
predicted = []
for word,tag in zip(p_sent,p_sent_tag):
    if word!='-00V-' :
        predicted.append((word,tag))
print(predicted)

In [ ]:
from keras.models import save_model

save_model(model,'withMasking.h5',include_optimizer=True)